<a href="https://colab.research.google.com/github/MarkLee7925/Bank-Marketing-Classification/blob/main/BankMarketing_DT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bank Marketing - Decision Tree

## Data preprocessing

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
# Load dataframe (df)
df = pd.read_csv('bank-additional-full.csv', sep=';')
X, y = df.iloc[:, :-1].values, df.iloc[:, -1].values

print(X.shape, y.shape)

(41188, 20) (41188,)


In [3]:
df

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,149,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,226,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,151,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,307,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,retired,married,professional.course,no,yes,no,cellular,nov,fri,334,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes
41184,46,blue-collar,married,professional.course,no,no,no,cellular,nov,fri,383,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41185,56,retired,married,university.degree,no,yes,no,cellular,nov,fri,189,2,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41186,44,technician,married,professional.course,no,no,no,cellular,nov,fri,442,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes


In [4]:
df.dtypes

age                 int64
job                object
marital            object
education          object
default            object
housing            object
loan               object
contact            object
month              object
day_of_week        object
duration            int64
campaign            int64
pdays               int64
previous            int64
poutcome           object
emp.var.rate      float64
cons.price.idx    float64
cons.conf.idx     float64
euribor3m         float64
nr.employed       float64
y                  object
dtype: object

In [5]:
df.describe()

,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
count,41188.00000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000
mean,40.02406,258.285010,2.567593,962.475454,0.172963,0.081886,93.575664,-40.502600,3.621291,5167.035911
std,10.42125,259.279249,2.770014,186.910907,0.494901,1.570960,0.578840,4.628198,1.734447,72.251528
min,17.00000,0.000000,1.000000,0.000000,0.000000,-3.400000,92.201000,-50.800000,0.634000,4963.600000
25%,32.00000,102.000000,1.000000,999.000000,0.000000,-1.800000,93.075000,-42.700000,1.344000,5099.100000
50%,38.00000,180.000000,2.000000,999.000000,0.000000,1.100000,93.749000,-41.800000,4.857000,5191.000000
75%,47.00000,319.000000,3.000000,999.000000,0.000000,1.400000,93.994000,-36.400000,4.961000,5228.100000
max,98.00000,4918.000000,56.000000,999.000000,7.000000,1.400000,94.767000,-26.900000,5.045000,5228.100000


In [6]:
# Check for missing values
df.isnull().sum()

age               0
job               0
marital           0
education         0
default           0
housing           0
loan              0
contact           0
month             0
day_of_week       0
duration          0
campaign          0
pdays             0
previous          0
poutcome          0
emp.var.rate      0
cons.price.idx    0
cons.conf.idx     0
euribor3m         0
nr.employed       0
y                 0
dtype: int64

In [7]:
# Categorical Columns (indices) - [1:9]
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
# le_columns = X[:, [1, 2, 3, 4, 5, 6, 7, 8, 9, 14]]

# for i in X[:]:
#   if i in le_columns:
#     X[:, i] = le.fit_transform(X[:, i])

X[:, 1] = le.fit_transform(X[:, 1])
X[:, 2] = le.fit_transform(X[:, 2])
X[:, 3] = le.fit_transform(X[:, 3])
X[:, 4] = le.fit_transform(X[:, 4])
X[:, 5] = le.fit_transform(X[:, 5])
X[:, 6] = le.fit_transform(X[:, 6])
X[:, 7] = le.fit_transform(X[:, 7])
X[:, 8] = le.fit_transform(X[:, 8])
X[:, 9] = le.fit_transform(X[:, 9])
X[:, 14] = le.fit_transform(X[:, 14])
y = le.fit_transform(y)

print(X[:, 1:14])
print(y)

[[3 1 0 ... 1 999 0]
 [7 1 3 ... 1 999 0]
 [7 1 3 ... 1 999 0]
 ...
 [5 1 6 ... 2 999 0]
 [9 1 5 ... 1 999 0]
 [5 1 5 ... 3 999 1]]
[0 0 0 ... 0 1 0]


In [8]:
# Split dataframe (df) into training and test sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, 
                                                    random_state=0)

In [9]:
# Feature Scale numerical columns
from sklearn.preprocessing import StandardScaler, MinMaxScaler

numerical_columns = [0, 10, 11, 12, 13, 15, 16, 17, 18, 19]
sc = StandardScaler()
X_train[:, numerical_columns] = sc.fit_transform(X_train[:, numerical_columns])
X_test[:, numerical_columns] = sc.transform(X_test[:, numerical_columns])

## Define RF model

In [10]:
from sklearn.tree import DecisionTreeClassifier

# Accuracy Scoring
clf = DecisionTreeClassifier(criterion='entropy', max_depth=6, 
                             min_samples_leaf=3, min_samples_split=2, 
                             random_state=4)

# ROC_AUC Scoring
# clf = DecisionTreeClassifier(criterion='entropy', max_depth=6, 
#                              min_samples_leaf=18, min_samples_split=2, 
#                              random_state=0)

# Train Model
clf.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=6, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=3, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=4, splitter='best')

## Evaluate model

In [11]:
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score, roc_auc_score

# Predict test set results
# If y_pred > 0.5, client has subscribed to a term deposit
y_pred = clf.predict(X_test)
y_pred = (y_pred > 0.5)
print('y_pred: y_test:')
print(np.concatenate((y_pred.reshape(len(y_pred), 1), y_test.reshape(len(y_test), 1)), 1 ))
print('')

# Create the confusion matrix
# TN = cm[0][0]  # True Negative
# FP = cm[0][1]  # False Positive
# FN = cm[1][0]  # False Negative
# TP = cm[1][1]  # True Positive
print('Confusion Matrix: ')
cm = confusion_matrix(y_test, y_pred)
print(cm)
print('')
print("Test Accuracy: %.3f%% " % (accuracy_score(y_test, y_pred) * 100))
# print("ROC_AUC: %.3f%% " % (accuracy_score(y_test, y_pred) * 100))
print('')

# accuracy = (TP + TN)/(TN + FP + FN + TP)
# precision = TP/(TP + FP)
# recall = TP/(TP + FN)
# f1_score = (2 * precision * recall)/(precision + recall)
# support = Number of occurrences of each class in y_pred
print('Classification Report: ')
print(classification_report(y_test, y_pred))


y_pred: y_test:
[[0 1]
 [0 0]
 [0 0]
 ...
 [1 1]
 [0 0]
 [0 0]]

Confusion Matrix: 
[[7095  224]
 [ 443  476]]

Test Accuracy: 91.903% 

Classification Report: 
              precision    recall  f1-score   support

           0       0.94      0.97      0.96      7319
           1       0.68      0.52      0.59       919

    accuracy                           0.92      8238
   macro avg       0.81      0.74      0.77      8238
weighted avg       0.91      0.92      0.91      8238



In [12]:
# Apply k-Fold Cross Validation on the training set
from sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold

cv = RepeatedStratifiedKFold(n_splits = 10, n_repeats = 3, random_state = 0)
scores = cross_val_score(clf, X_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1)
# scores = cross_val_score(clf, X_train, y_train, scoring='roc_auc', cv=cv, n_jobs=-1)

# Force scores to be positive
scores = abs(scores)
print("Mean Accuracy: %.3f%%" % (np.mean(scores) * 100))
print("Standard Deviation: %.3f%%" % np.std(scores))
print("Min Accuracy: %.3f%%" % (np.min(scores) * 100))
print("Max Accuracy: %.3f%%" % (np.max(scores) * 100))

Mean Accuracy: 91.392%
Standard Deviation: 0.004%
Min Accuracy: 90.501%
Max Accuracy: 92.170%


<table border = "1">
  <thead>
    <th colspan = "3">Cross-Validation Results</th>
  </thead>
  <tbody>
    <tr>
      <td><b></b></td>
      <td><b>Accuracy</b></td>
      <td><b>ROC_AUC</b></td>
    </tr>
    <tr>
      <td><b>Mean</b></td>
      <td>91.392%</td>
      <td>93.575%</td>
    </tr>
    <tr>
      <td><b>Std.</b></td>
      <td>0.004%</td>
      <td>0.006%</td>
    </tr>
    <tr>
      <td><b>Min</b></td>
      <td>90.501%</td>
      <td>91.603%</td>
    </tr>
    <tr>
      <td><b>Max</b></td>
      <td>92.170%</td>
      <td>94.683%</td>
    </tr>
  </tbody>
</table>

In [13]:
# Apply Grid Search for parameter tuning
from sklearn.model_selection import GridSearchCV

# param_dict = {'criterion': ['gini', 'entropy'],
#               'max_depth': [5, 10, 25, 50],
#               'min_samples_leaf': [2, 5, 10],
#               'min_samples_split': [2, 5, 10],
#               'random_state': [0, 1, 2]}

# Accuracy scoring
param_dict = {'criterion': ['entropy'],
              'max_depth': [6],
              'min_samples_leaf': [2, 3],
              'min_samples_split': [2, 3],
              'random_state': [2, 3, 4]}

# ROC_AUC scoring
# param_dict = {'criterion': ['entropy'],
#               'max_depth': [6],
#               'min_samples_leaf': [17, 18, 19],
#               'min_samples_split': [1, 2, 3],
#               'random_state': [0, 1]}

grid = GridSearchCV(clf, param_dict, scoring='accuracy', cv=cv, n_jobs=-1)
# grid = GridSearchCV(clf, param_dict, scoring='roc_auc', cv=cv, n_jobs=-1)
results = grid.fit(X_train, y_train)

# Summarize results
best_accuracy = grid.best_score_
best_parameters = grid.best_params_
print("Best Accuracy: %.3f%%" % (best_accuracy * 100))
print("Best Parameters: ", best_parameters)


Best Accuracy: 91.392%
Best Parameters:  {'criterion': 'entropy', 'max_depth': 6, 'min_samples_leaf': 3, 'min_samples_split': 2, 'random_state': 3}


<table border = "1">
  <thead>
    <th colspan = "3">Grid Search Results</th>
  </thead>
  <tbody>
    <tr>
      <td><b></b></td>
      <td><b>Accuracy</b></td>
      <td><b>ROC_AUC</b></td>
    </tr>
    <tr>
      <td><b>Best Score</b></td>
      <td>91.392%</td>
      <td>93.575%</td>
    </tr>
    <tr>
      <td colspan="3"><b>Best Parameters:</b></td>
    </tr>
    <tr>
      <td><b>criterion</b></td>
      <td>'entropy'</td>
      <td>'entropy'</td>
    </tr>
    <tr>
      <td><b>max_depth</b></td>
      <td>6</td>
      <td>6</td>
    </tr>
    <tr>
      <td><b>min_samples_leaf</b></td>
      <td>3</td>
      <td>18</td>
    </tr>
    <tr>
      <td><b>min_samples_split</b></td>
      <td>2</td>
      <td>2</td>
    </tr>
    <tr>
      <td><b>random_state</b></td>
      <td>3</td>
      <td>0</td>
    </tr>
  </tbody>
</table>